<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Annotate_Images_Part2.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


## Automatic Face Image Annotation & Masked Photo Generation ##

In the previous notebook, we prepared the unmasked photos in the dataset. In this notebook we will look at pre-processing the images for our masked photo dataset.

Firstly we will identify the bounding boxes for the faces as with the unmasked photos but additionally we will then also place a series of masked into those faces thus making them masked photos.

### Import required libraries

In [1]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import * 
!wget {ANNOTATE}
!wget {PLACE_MASKS}

!pip install face_recognition
!pip install wget

from annotate import *
from place_masks import * 

import os
import glob
import shutil
import tarfile

import sys
import random

--2020-06-10 18:42:28--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4376 (4.3K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   4.27K  --.-KB/s    in 0s      

2020-06-10 18:42:29 (56.8 MB/s) - ‘constants.py’ saved [4376/4376]

--2020-06-10 18:42:30--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/annotate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2658 (2

### Prepare dataset ###

#### Connect to Google Drive


In [0]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


#### Download faces dataset and extract to folder

In [2]:
os.chdir(DATASET_DIR_TMP1)
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/1k_faces_01.zip
!unzip -j 1k_faces_01.zip '*.jpg'
!rm 1k_faces_01.zip


--2020-06-10 18:44:00--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/1k_faces_01.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23768454 (23M) [application/zip]
Saving to: ‘1k_faces_01.zip’

1k_faces_01.zip     100%[===================>]  22.67M  6.72MB/s    in 4.1s    

2020-06-10 18:44:04 (5.58 MB/s) - ‘1k_faces_01.zip’ saved [23768454/23768454]

Archive:  1k_faces_01.zip
  inflating: 00SD82OK2A.jpg          
  inflating: 00V5CZZSSO.jpg          
  inflating: 01P734HDO0.jpg          
  inflating: 02IWA7SQR9.jpg          
  inflating: 02PO4NZNKS.jpg          
  inflating: 02VVQ8EUPO.jpg          
  inflating: 0393CKG05Z.jpg          
  inflating: 04W6LHK65S.jpg          
  inflating: 062U7MAFP7.jpg          
  inflating

#### Annotate the images
Calls the annotate function to automatically annotate the faces in our image folder and outputs XML files

In [0]:
annotate(DATASET_DIR_TMP1,DATASET_DIR_TMP2)

## Generate Masked Photos

### Download required dependency and extract to folder

In [0]:
os.chdir(DATASET_DIR_UNPREP)
tf = tarfile.open(wget.download(MASKS))
tf.extractall()


### Define environment

In [6]:
model = "hog" # Select facial recognition model; Available options areh 'hog' or 'cnn'. Hog is faster, cnn is more accurate.

input_folder_list = [os.path.join(DATASET_DIR_TMP1, f) for f in os.listdir(DATASET_DIR_TMP1) if os.path.isfile(os.path.join(DATASET_DIR_TMP1, f))]
mask_folder_list = [os.path.join(DATASET_DIR_UNPREP_MASKS, f) for f in os.listdir(DATASET_DIR_UNPREP_MASKS) if os.path.isfile(os.path.join(DATASET_DIR_UNPREP_MASKS, f))]

for i in range(len(input_folder_list)):
     # get a random mask from from the mask folder
     random_mask_file = random.choice(list(mask_folder_list))
        
     if random_mask_file.endswith(('.png', '.jpg', '.jpeg')): 
      # get the next image from the image folder
      current_image_file = input_folder_list[i]
      if current_image_file.endswith(('.png', '.jpg', '.jpeg')):
        create_mask(current_image_file, random_mask_file, model)


Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/RUU2Q2AHNQ.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/9DOX8YVCTM.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/OJNK27MTVT.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/1J6OOYEGMT.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/7D6GXT9VOH.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/9SHITYUZM2.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/5TVFA7WXE6.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/CQLRBJBVWD.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/YN42WR2FOD.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/C29YVSEIQQ.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detection/dataset/tmp1/B7QWCGZQ24.jpg
Save to /content/tensorflow/workspace/Face-Mask-Detect

### Helper code to place masks on the faces

#### We loop through all the files in our input directory, place masks on the faces and the save them in the output directory

#### Save the generated images and archive them

In [0]:
def move_files(source,destination):
    files = os.listdir(source)
    for f in files:
        if f.endswith(('.png', '.jpg', '.jpeg', '.xml')):
          shutil.move(source+"/"+f, destination+"/")

# Move the images and the annotations from the temp 
move_files(DATASET_DIR_TMP1,DATASET_DIR_UNPREP_IMG)
move_files(DATASET_DIR_TMP2,DATASET_DIR_UNPREP_ANNO)

### Compress Dataset and Save Dataset to Google Drive

Follow this step if you wish to archive your dataset for future use

In [0]:
!zip -r masked-datasetv2.zip {DATASET_DIR_UNPREP}
!gsutil cp masked-datasetv2.zip {DRIVE_DEV}

  adding: content/training_demo/images/ (stored 0%)
  adding: content/training_demo/images/EIZSKIRIT6.xml (deflated 47%)
  adding: content/training_demo/images/WHHC578I6K.jpg (deflated 1%)
  adding: content/training_demo/images/M5XU02T5UA.xml (deflated 47%)
  adding: content/training_demo/images/XJ1BCWDGBX.xml (deflated 47%)
  adding: content/training_demo/images/XCYCNCMDIF.jpg (deflated 1%)
  adding: content/training_demo/images/UBLHJMVTA7.xml (deflated 47%)
  adding: content/training_demo/images/GAW0VO7OTB.jpg (deflated 1%)
  adding: content/training_demo/images/T5AD07K975.jpg (deflated 1%)
  adding: content/training_demo/images/0PKNKA4WR6.jpg (deflated 1%)
  adding: content/training_demo/images/E4ZTFZD4MW.jpg (deflated 1%)
  adding: content/training_demo/images/8N6KGKVX2L.xml (deflated 47%)
  adding: content/training_demo/images/9NU2BBTWZW.jpg (deflated 1%)
  adding: content/training_demo/images/VQ0PNYDJV9.xml (deflated 46%)
  adding: content/training_demo/images/KLQ35JH2NE.xml (def